# Modelos Preditivos de Matemática Aplicada a Ciência de Dados

## Modelos Utilizados

### Classificação

- GaussianNB
- DecisionTreeClassifier
- KNeighborsClassifier
- RandomForestClassifier

### Regressão

- LinearRegression
- DecisionTreeRegressor
- KNeighborsRegressor
- RandomForestRegressor

### Link do dataset: https://www.kaggle.com/datasets/andriyaniarimbi/predictive-maintenance

Obs: Neste projeto temos três datasets, porém para este notebook estamos testando somente o predictive_maintence que verifica se é necessário ou não realizar a manutenção de um ambiente.

# Imports

In [38]:
## Modelos

### Modelos de classificação
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

### Modelos de Regressão
import numpy as np
from sklearn.linear_model import LinearRegression

## Processamento e normalização de dados
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler

## Testes e Avaliação
from sklearn.model_selection import train_test_split, GridSearchCV

## Métricas
from sklearn.metrics import accuracy_score, r2_score,precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

from sklearn.base import is_classifier, is_regressor

## Serialização
import redis
import joblib
import os


## Processo de carregar e filtrar os dados do dataset

In [39]:
# Carregar o dataset em um dataframe
df = pd.read_csv('./datasets/predictive_maintenance.csv')

# Deletar colunas que não são uteis
df_processed = df.drop(['UDI', 'Product ID', 'Target'], axis=1)

# Normalização da coluna Failure Type para número
label_encoder = LabelEncoder()
df_processed['Failure Type'] = label_encoder.fit_transform(df_processed['Failure Type'])

# A coluna 'Type' é uma feature. Vamos usar a codificação one-hot para converter
# Isso impede que o modelo pense que 'L' < 'M' < 'H'.
df_processed = pd.get_dummies(df_processed, columns=['Type'], drop_first=True)

# Separando as colunas preditada e o resto
X = df_processed.drop('Failure Type', axis=1)
y = df_processed['Failure Type']

print("\nProcessed Features (X) head:")
print(X.head())


Processed Features (X) head:
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   
3                298.2                    308.6                    1433   
4                298.2                    308.7                    1408   

   Torque [Nm]  Tool wear [min]  Type_L  Type_M  
0         42.8                0   False    True  
1         46.3                3    True   False  
2         49.4                5    True   False  
3         39.5                7    True   False  
4         40.0                9    True   False  


In [40]:
#Vizualiação do dataframe 
display(df_processed)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,1,False,True
1,298.2,308.7,1408,46.3,3,1,True,False
2,298.1,308.5,1498,49.4,5,1,True,False
3,298.2,308.6,1433,39.5,7,1,True,False
4,298.2,308.7,1408,40.0,9,1,True,False
...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,1,False,True
9996,298.9,308.4,1632,31.8,17,1,False,False
9997,299.0,308.6,1645,33.4,22,1,False,True
9998,299.0,308.7,1408,48.5,25,1,False,False


In [41]:
# Definindo valores únicos da coluna que vai ser prevista 
df_processed['Failure Type'].unique()

array([1, 3, 5, 2, 4, 0])

In [42]:
#Definindo dados de teste e dados de treino 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nQuantidade de linhas de Treinamento: {x_train.shape}")
print(f"\nQuantidade de linhas de Teste: {x_test.shape}")


Quantidade de linhas de Treinamento: (8000, 7)

Quantidade de linhas de Teste: (2000, 7)


In [43]:
# Colunas que são numericas e precisam de normalização
numerical_features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

#Inicializar o scaler
scaler = StandardScaler()

# Colocar o scaler para aprender os parâmetros de normalização (média e desvio padrão) a partir dos dados de treino
scaler.fit(x_train[numerical_features])

# Transoformar os dados de treino e teste
x_train[numerical_features] = scaler.transform(x_train[numerical_features])
x_test[numerical_features] = scaler.transform(x_test[numerical_features])

print("\nX_train normalizado:")
print(x_train.head())


X_train normalizado:
      Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
9254            -0.854066                -0.609589                0.427634   
1561            -0.904014                -1.080528               -0.834945   
1670            -0.904014                -1.484190               -0.059677   
6087             0.444571                 0.534121                0.333495   
6669             0.694309                 0.332290                0.178441   

      Torque [Nm]  Tool wear [min]  Type_L  Type_M  
9254    -0.892696         1.375035    True   False  
1561     1.382187         0.457620    True   False  
1670    -0.892696         1.359218    True   False  
6087    -0.702288        -1.598655   False    True  
6669    -0.612094         1.580663    True   False  


## Classificação do modelo


In [44]:
# --- Determinar o tipo de problema ---
# Número de valores distintos na variável alvo
unique_values = y.nunique()
print(f"\nValores únicos na variável alvo: {unique_values}")
print(f"Exemplos de valores: {y.unique()[:10]}")

# Verifica o tipo de problema
if y.dtype == 'object' or unique_values < 20:
    problema = "classificacao"
else:
    problema = "regressao"

print(f"\nTipo de problema detectado: {problema.upper()}")



Valores únicos na variável alvo: 6
Exemplos de valores: [1 3 5 2 4 0]

Tipo de problema detectado: CLASSIFICACAO


# Modelos


### Justificativa dos paramêtros para validação Cruzada

### Modelos de Classificação
#### GaussianNB
- O var_smoothing evita divisões por 0 na hora de calcular probabilidade e adiciona um valor de variância.
- A escolha do np.logspace é porque o intervalo descrito cobre uma boa faixa logarítmica visto que a função é muito sensível a números pequenos e com uma escala logarítmica ela fica mais estável e garante melhores resultados.

#### DecisionTreeClassifier
- Crtitérios: Gini e Entropia, ambos são modelos pra prever uma coluna de forma de classificação.

- max_depth: limita a profundidade da árvore.
    - Valores pequenos evitam overfitting, mas podem gerar underfitting.
    - None permite crescer até o limite dos dados (controle total).
    - 10, 20, 30 são escolhas típicas de compromisso entre precisão e generalização.

- min_samples_split: número mínimo de amostras para dividir um nó.
    - [2, 5, 10] cobre desde divisão agressiva (2) até árvores mais generalistas.

#### KNNClassifier
- n_neighbors: número de vizinhos para votar.
    - [3, 5, 7, 9, 11] cobre casos de baixo a médio número de vizinhos.
    - Valores muito grandes diluem as fronteiras de decisão e resultam na perda de precisão.

- weights:
    - "uniform": todos vizinhos têm peso igual.
    - "distance": vizinhos mais próximos têm mais influência → tende a melhorar modelos com ruído.

- metric:
    - "euclidean": padrão em dados contínuos.
    - "manhattan": mais robusto quando há outliers.

#### RandomForestClassifier
- n_estimators: número de árvores.
    - [100, 200] é suficiente para estabilizar o modelo sem sobrecarregar o treino.

- max_depth, min_samples_split, min_samples_leaf, max_features, criterion:
    - Mesmos significados e motivações do DecisionTree, mas aqui são testados em conjunto dentro do ensemble.

### Modelos de Regressão

#### LinearRegression

- fit_intercept: define se o modelo calcula o termo b (intercepto).
    - True: padrão, útil quando os dados não estão centrados.
    - False: útil se você já normalizou ou centralizou os dados.

- copy_X: se copia ou não o array original.
    - False economiza memória (útil em datasets grandes).
    - True é mais seguro (não modifica os dados originais).

#### RandomForestRegressor

- Mesmas justificativas do RandomForestClassifier.
- O criterion = 'squared_error' é padrão para regressão, pois mede o erro médio quadrático entre previsão e real.

#### KNeighborsRegressor

- n_neighbors, weights, metric tem as mesmas ideias que no KNNClassifier.

- algorithm: define o método de busca de vizinhos.
    - "auto": escolhe automaticamente.
    - "ball_tree" / "kd_tree": eficientes em dados numéricos.
    - "brute": busca exata (mais lenta, mas serve de controle).

- leaf_size: controla a granularidade das árvores internas (afeta tempo x precisão).

- p:
    - 1 é a distância de Manhattan.
    - 2 é a distância Euclidiana (padrão).

#### DecisionTreeRegressor

- Igual ao DecisionTreeClassifier, mas o criterion muda:
- squared_error → mede a soma dos erros quadráticos (regressão).

## Modelos

In [45]:
if problema == "classificacao": #Se o problema for classificação
    models = {
        "GaussianNB": (
            GaussianNB(),
            {"var_smoothing": np.logspace(0, -9, num=100)}
        ),
        "DecisionTreeClassifier": (
            DecisionTreeClassifier(random_state=42),
            {
                "criterion": ["gini", "entropy"],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
                "max_features": ["sqrt", "log2", None]
            }
        ),
        "KNNClassifier": (
            KNeighborsClassifier(),
            {
                "n_neighbors": [3, 5, 7, 9, 11],
                "weights": ["uniform", "distance"],
                "metric": ["euclidean", "manhattan"]
            }
        ),
        "RandomForestClassifier": (
            RandomForestClassifier(),
            {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
            }
        )
    }

else:  #Se problema for regressão
    models = {
        "LinearRegression": (
            LinearRegression(),
            {
                'fit_intercept': [True, False],
                'copy_X': [True, False]
            }
        ),
        "RandomForestRegressor": (
            RandomForestRegressor(),
            {
                'n_estimators': [100,200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['sqrt', 'log2'],  
                'criterion': ['squared_error']  
            }
        ),
        "KNeighborsRegressor":(
            KNeighborsRegressor(),
            {
                'n_neighbors': [3,5,7,9,11],
                'weights': ['uniform','distance'],
                'algorithm': ['auto','ball_tree','kd_tree','brute'],
                'leaf_size': [20,30,40,50],
                'p': [1,2],
                'metric': ['minkowski','euclidean','manhattan']
            }
        ),
        "DecisionTreeRegressor":(
            DecisionTreeRegressor(),
            {
                "criterion": ["squared_error"],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
                "max_features": ["sqrt", "log2", None]
            }
        )
    }


# Dicionários para armazenar resultados
best_models = {}
metrics = {}

# Loop para treinar e avaliar cada modelo
for name, (model, params) in models.items():
    print(f"\nTreinando {name}...")

    # Detecta automaticamente o tipo do modelo
    modelo_tipo = "regressor" if is_regressor(model) else "classificador"

    # Define a métrica de pontuação correta
    scoring_metric = 'r2' if modelo_tipo == "regressor" else 'accuracy'

    gs = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=scoring_metric,
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(x_train, y_train)

    print(f"Melhores parâmetros para {name}: {gs.best_params_}")

    best_model = gs.best_estimator_
    best_models[name] = best_model

    # prepara dicionário com todas as chaves (garante consistência entre modelos)
    model_metrics = {
        # Métricas de classificação
        "Accuracy": np.nan,
        "Precision": np.nan,
        "Recall": np.nan,
        "F1-Score": np.nan,
        "Confusion Matrix": None,
        "Best Params": None,
        # Métricas de regressão
        "R2": np.nan,
        "MSE": np.nan,
        "RMSE": np.nan,
        "MAE": np.nan,
    }

    # Salva os melhores parâmetros e score cross-val do GridSearch
    model_metrics["Best Params"] = gs.best_params_
    model_metrics["CV Score"] = gs.best_score_

    # Previsões no conjunto de teste
    y_pred = best_model.predict(x_test)

    # Se for regressão, calcula métricas de regressão (e opcionalmente converte para classif.)
    if modelo_tipo == "regressor":
        # métricas de regressão contínuas
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        model_metrics["R2"] = r2
        model_metrics["MSE"] = mse
        model_metrics["RMSE"] = rmse
        model_metrics["MAE"] = mae

        # mantém também as métricas de classificação caso queira comparar (conversão por arredondamento)
        y_pred_labels = np.rint(y_pred).astype(int)
        y_pred_labels = np.clip(y_pred_labels, int(y_test.min()), int(y_test.max()))

    else:
        # Classificação padrão
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

        model_metrics["Accuracy"] = acc
        model_metrics["Precision"] = prec
        model_metrics["Recall"] = rec
        model_metrics["F1-Score"] = f1
        model_metrics["Confusion Matrix"] = cm

    # salva métricas padronizadas
    metrics[name] = model_metrics

    # Serialização dos modelos
    metrics[name] = model_metrics

    # Serialização dos modelos (salva em ./pickles e opcionalmente envia para Redis)
    pickle_dir = "pickles"

    # verifica se a pasta existe antes de criar
    if not os.path.isdir(pickle_dir):
        os.makedirs(pickle_dir)

    # salva o ESTIMADOR ÓTIMO (best_model) com nome do modelo
    pickle_path = os.path.join(pickle_dir, f"{name}.pkl")
    joblib.dump(best_model, pickle_path)

    # tenta enviar para redis (chave por modelo); captura erro caso o redis não esteja disponível
    try:
        r = redis.Redis(host="localhost", port=6379, db=0, decode_responses=False)
        with open(pickle_path, "rb") as f:
            r.set(f"ml_model:{name}".encode(), f.read())
        print("Adicionado ao Redis com sucesso!")
    except Exception as e:
        print(f"Aviso: não foi possível salvar {name} no Redis: {e}")


# Exibe resumo geral das métricas (última célula: separação por tipo)
summary_df = pd.DataFrame(metrics).T



Treinando GaussianNB...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Melhores parâmetros para GaussianNB: {'var_smoothing': np.float64(1.0)}
Adicionado ao Redis com sucesso!

Treinando DecisionTreeClassifier...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Melhores parâmetros para DecisionTreeClassifier: {'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Adicionado ao Redis com sucesso!

Treinando KNNClassifier...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros para KNNClassifier: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Adicionado ao Redis com sucesso!

Treinando RandomForestClassifier...
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Melhores parâmetros para RandomForestClassifier: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Adicion

# Métricas de Avaliação

In [46]:
# Garante que 'Best Params' é sempre um dicionário
if 'Best Params' in summary_df.columns:
    summary_df['Best Params'] = summary_df['Best Params'].apply(
        lambda d: d if isinstance(d, dict) else {}
    )

# Detecta linhas de regressão/classificação
if 'R2' in summary_df.columns:
    is_regressor = summary_df['R2'].notna()
else:
    # Detecta regressão pela presença de métricas de erro
    regression_metrics = summary_df.filter(regex='MSE|RMSE|MAE|R2')
    is_regressor = regression_metrics.notna().any(axis=1)

# Diz que o modelo é classificador quando não é regressor e o inverso também
is_classifier = is_regressor == False

# Colunas relevantes
regression_cols = ["R2", "MSE", "RMSE", "MAE", "Best Params"]
classification_cols = [
    "Accuracy", "Precision", "Recall", "F1-Score",
    "Confusion Matrix", "Best Params"
]

# Exibição de resultados
print("\nResumo - Regressão:\n")
if is_regressor.any():
    cols = [c for c in regression_cols if c in summary_df.columns]
    display(summary_df.loc[is_regressor, cols])
else:
    print("Nenhum modelo de regressão executado.")

print("\nResumo - Classificação:\n")
if is_classifier.any():
    cols = [c for c in classification_cols if c in summary_df.columns]
    display(summary_df.loc[is_classifier, cols])
else:
    print("Nenhum modelo de classificação executado.")



Resumo - Regressão:

Nenhum modelo de regressão executado.

Resumo - Classificação:



,Accuracy,Precision,Recall,F1-Score,Confusion Matrix,Best Params
GaussianNB,0.969,0.945516,0.969,0.955933,"[[0, 15, 0, 0, 0, 0], [0, 1933, 0, 2, 0, 0], [...",{'var_smoothing': 1.0}
DecisionTreeClassifier,0.9755,0.969122,0.9755,0.972152,"[[13, 2, 0, 0, 0, 0], [7, 1916, 5, 4, 0, 3], [...","{'criterion': 'gini', 'max_depth': 10, 'max_fe..."
KNNClassifier,0.9715,0.96424,0.9715,0.964424,"[[4, 11, 0, 0, 0, 0], [3, 1928, 2, 2, 0, 0], [...","{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
RandomForestClassifier,0.9815,0.972816,0.9815,0.976714,"[[10, 5, 0, 0, 0, 0], [1, 1930, 0, 4, 0, 0], [...","{'criterion': 'gini', 'max_depth': None, 'max_..."


##